Libraries

In [1]:
import tensorflow as tf
from   tensorflow.keras.models import Model
from   tensorflow.keras import layers
import matplotlib.pyplot as plt
from   tensorflow.keras.applications import EfficientNetB3, EfficientNetB0, EfficientNetB1
from   tensorflow.keras.layers import Conv2D
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import EfficientNetB3, ResNet50, MobileNetV3Large
from tensorflow.keras.layers import Conv2D, Multiply, Reshape, Input, GlobalMaxPooling2D, Add, BatchNormalization, GlobalAveragePooling2D, Dense
from tensorflow_addons.optimizers import AdamW
import math
import numpy as np
import os

c:\Users\39351\anaconda3\envs\TFgpuKeras\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.6.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.6.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


Parameters and Hyperparameters settings

In [2]:
# Load and preprocess the data
batch_size = 256
img_height = 32
img_width = 32
IMG_SIZE = img_height
epochs = 12
num_classes = 3

In [3]:
def se_block(input_tensor, reduction=16):
    channel_axis = -1
    filters = input_tensor.shape[channel_axis]

    # Channel-wise attention (SE Block)
    se = GlobalAveragePooling2D()(input_tensor)
    se = Reshape((1, 1, filters))(se)
    se = Dense(filters // reduction, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)
    se = Multiply()([input_tensor, se])

    # Spatial attention
    spatial = Conv2D(filters // reduction, (1, 1), activation='relu', padding='same', kernel_initializer='he_normal', use_bias=False)(input_tensor)
    spatial = BatchNormalization()(spatial)
    spatial = Conv2D(1, (1, 1), activation='sigmoid', padding='same', kernel_initializer='he_normal', use_bias=False)(spatial)

    # Apply spatial attention
    spatial = Multiply()([input_tensor, spatial])

    # Combine channel-wise and spatial attentions
    x = Add()([input_tensor, se, spatial])
    return x

# Convolutional Block Attention Module (CBAM)
def cbam_block(input_tensor, reduction=16):
    channel = input_tensor.shape[-1]
    
    # Channel attention
    avg_pool = GlobalAveragePooling2D()(input_tensor)
    max_pool = GlobalMaxPooling2D()(input_tensor)
    avg_pool = Reshape((1, 1, channel))(avg_pool)
    max_pool = Reshape((1, 1, channel))(max_pool)
    dense1 = Dense(channel // reduction, activation='relu', kernel_initializer='he_normal', use_bias=False)
    dense2 = Dense(channel, kernel_initializer='he_normal', use_bias=False)
    avg_out = dense2(dense1(avg_pool))
    max_out = dense2(dense1(max_pool))
    channel_attention = layers.Add()([avg_out, max_out])
    channel_attention = layers.Activation('sigmoid')(channel_attention)
    x = Multiply()([input_tensor, channel_attention])
    
    # Spatial attention
    avg_pool = tf.reduce_mean(x, axis=-1, keepdims=True)
    max_pool = tf.reduce_max(x, axis=-1, keepdims=True)
    concat = layers.Concatenate(axis=-1)([avg_pool, max_pool])
    spatial_attention = Conv2D(1, (7, 7),dilation_rate= 2, padding='same', activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(concat)
    x = Multiply()([x, spatial_attention])
    
    return x

class TransformerBlock(layers.Layer):
    def __init__(self, num_heads, embed_dim, ff_dim, rate=0.1, **kwargs):
        super().__init__(**kwargs)
        self.num_heads = num_heads
        self.embed_dim = embed_dim
        self.ff_dim = ff_dim
        self.rate = rate

        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)
        
    def get_config(self):
        config = super().get_config()
        config.update({
            'num_heads': self.num_heads,
            'embed_dim': self.embed_dim,
            'ff_dim': self.ff_dim,
            'rate': self.rate
        })
        return config

def create_model(img_height=224, img_width=224, num_classes=4):
    inputs = Input(shape=(img_height, img_width, 3))

    # First EfficientNetB3 instance
    efficient_net_b3_1 = EfficientNetB3(weights="imagenet", include_top=False, input_tensor=inputs)
    #efficient_net_b3_1.summary()
    x = efficient_net_b3_1.output

    # Pyramid module with SE blocks and CBAM
    c5 = se_block(x)
    p4 = Conv2D(filters=64, kernel_size=(1, 1), activation='relu')(c5)
    p4 = layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(p4)

    # Second EfficientNetB3 instance
    c4 = efficient_net_b3_1.get_layer('block4a_expand_activation').output
    c4 = cbam_block(c4)
    c4 = Conv2D(filters=64, kernel_size=(1, 1), activation='relu')(c4)
    c4 = layers.Lambda(lambda x: tf.image.resize(x, (2, 2)))(c4)
    p4 = layers.Concatenate()([p4, c4])

    p3 = Conv2D(filters=64, kernel_size=(1, 1), dilation_rate=(2, 2), activation='relu')(p4)
    p3 = layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(p3)

    # Third EfficientNetB3 instance
    c3 = efficient_net_b3_1.get_layer('block3a_expand_activation').output
    c3 = cbam_block(c3)
    c3 = Conv2D(filters=64, kernel_size=(1, 1), activation='relu')(c3)
    c3 = layers.Lambda(lambda x: tf.image.resize(x, (4, 4)))(c3)
    p3 = layers.Concatenate()([p3, c3])

    x = TransformerBlock(num_heads=4, embed_dim=128, ff_dim=96, rate=0.3)(p3)
    x = layers.Reshape((int(x.shape[1]), int(x.shape[2]), int(x.shape[3])))(x)
    x_c1 = layers.Conv2D(64, (3, 3), padding='same', dilation_rate=(2, 2), activation='relu')(x)
    x_c01 = TransformerBlock(num_heads=4, embed_dim=64, ff_dim=64, rate=0.3)(x_c1)
    x_c001 = layers.Reshape((int(x_c01.shape[1]), int(x_c01.shape[2]), int(x_c01.shape[3])))(x_c01)
    x_c11 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(x_c001)
    x_c11 = layers.BatchNormalization()(x_c11)
    x_c2 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(x_c11)
    combined1 = layers.Concatenate()([x_c1, x_c2])
    x = layers.MaxPooling2D()(combined1)
    x = layers.Flatten()(x)

    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dense(96, activation='relu', name="features_vector")(x)

    if num_classes == 2:
        x = layers.Dense(1, activation='sigmoid', name="pred")(x)
    else:
        x = layers.Dense(num_classes, activation='softmax')(x)

    model_created = Model(inputs=inputs, outputs=x)
    return model_created

def create_base_model(model_name, img_height=32, img_width=32, num_classes=4):
    if model_name == 'ResNet50':
        base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(img_height, img_width, 3))
    elif model_name == 'MobileNetV3Large':
        base_model = MobileNetV3Large(include_top=False, weights='imagenet', input_shape=(img_height, img_width, 3))
    elif model_name == 'EfficientNetB3':
        base_model = EfficientNetB3(include_top=False, weights='imagenet', input_shape=(img_height, img_width, 3))
    else:
        raise ValueError("Model name not recognized. Choose from 'ResNet50', 'MobileNetV3Large', or 'EfficientNetB3'.")

    # Add custom classification head
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    # Define the model
    model = Model(inputs=base_model.input, outputs=predictions)

    return model

In [ ]:
# Initialize the AdamW optimizer
optimizer = AdamW(learning_rate=0.001, weight_decay=1e-4)

def advanced_schedule(epoch, initial_lr=1e-4, warmup_epochs=5, total_epochs=10, min_lr=1e-6):
    """
    Learning rate schedule with warmup and cosine annealing.

    Parameters:
    - epoch: The current epoch.
    - initial_lr: Learning rate at the start of the warmup period.
    - warmup_epochs: Number of epochs to linearly increase the learning rate.
    - total_epochs: Total number of epochs for the cosine annealing schedule.
    - min_lr: Minimum learning rate to avoid reducing the learning rate too much.

    Returns:
    - lr: The learning rate for the current epoch.
    """
    if epoch < warmup_epochs:
        # Linear warmup
        lr = initial_lr * (epoch / warmup_epochs)
    else:
        # Cosine annealing after warmup
        effective_epoch = epoch - warmup_epochs
        progress = effective_epoch / (total_epochs - warmup_epochs)
        lr = initial_lr * 0.5 * (1 + math.cos(math.pi * progress))
    
    # Ensure learning rate does not drop below minimum value
    lr = max(lr, min_lr)
    
    return lr
advanced_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda epoch: advanced_schedule(epoch))

# Function to recursively gather image paths and labels from all categories, grouped by patient
def gather_image_paths_and_labels(data_dir):
    categories = ['01_Fat', '02_Stroma', '03_Tumor']
    patient_image_paths = {}
    label_map = {category: idx for idx, category in enumerate(categories)}
    valid_image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']  # Add other valid image extensions if necessary
    
    for category in categories:
        category_path = os.path.join(data_dir, category)
        for patient_folder in sorted(os.listdir(category_path)):
            patient_folder_path = os.path.join(category_path, patient_folder)
            if os.path.isdir(patient_folder_path):
                patient_id = os.path.join(category, patient_folder)
                if patient_id not in patient_image_paths:
                    patient_image_paths[patient_id] = {'image_paths': [], 'label': label_map[category]}
                # Recursively search for image files
                for root, _, files in os.walk(patient_folder_path):
                    for file in files:
                        if any(file.lower().endswith(ext) for ext in valid_image_extensions):
                            patient_image_paths[patient_id]['image_paths'].append(os.path.join(root, file))
    
    return patient_image_paths

# Modify the `create_dataset` function to include sample weights
def create_dataset(image_paths, labels, sample_weights=None, image_size=(224, 224), batch_size=32, one_hot=False, num_classes=3):
    ds = []
    path_ds = tf.data.Dataset.from_tensor_slices(image_paths)
    image_ds = path_ds.map(lambda x: tf.image.resize(tf.image.decode_jpeg(tf.io.read_file(x), channels=3), image_size))

    if one_hot:
        labels = tf.keras.utils.to_categorical(labels, num_classes=num_classes)
    
    label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(labels, tf.float32))
    
    if sample_weights is not None:
        sample_weight_ds = tf.data.Dataset.from_tensor_slices(tf.cast(sample_weights, tf.float32))
        ds = tf.data.Dataset.zip((image_ds, label_ds, sample_weight_ds))
    else:
        ds = tf.data.Dataset.zip((image_ds, label_ds))

    ds = ds.shuffle(buffer_size=len(image_paths))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)

    return ds

# Function to count images per category
def count_images_per_category(image_paths, labels, categories):
    category_counts = {category: 0 for category in categories}
    for label in labels:
        category = categories[label]
        category_counts[category] += 1
    return category_counts


def evaluate_metrics(val_true, val_pred):
    cm = confusion_matrix(val_true, val_pred)
    #print("Confusion Matrix:\n", cm)
    
    sensitivity = sensitivity_multiclass(cm)
    specificity = specificity_multiclass(cm)
    f1 = f1_score_multiclass(cm)
    accuracy = accuracy_multiclass(cm)
    iou = iou_multiclass(cm)
    
    '''print(f"Sensitivity: {sensitivity}")
    print(f"Specificity: {specificity}")
    print(f"F1 Score: {f1}")
    print(f"Accuracy: {accuracy}")
    print(f"IoU: {iou}")'''

    # Detailed classification report
    #print("\nClassification Report:\n", classification_report(val_true, val_pred))
    
class MetricsCallback(tf.keras.callbacks.Callback):
    def __init__(self, val_data):
        super().__init__()
        self.val_data = val_data
        self.sensitivities = []
        self.specificities = []
        self.f1_scores = []
        self.accuracies = []
        self.ious = []

    def on_epoch_end(self, epoch, logs=None):
        val_true = []
        val_pred = []

        for images, labels in self.val_data:
            preds = self.model.predict(images)
            val_true.extend(np.argmax(labels.numpy(), axis=1))
            val_pred.extend(np.argmax(preds, axis=1))

        val_true = np.array(val_true)
        val_pred = np.array(val_pred)

        cm = confusion_matrix(val_true, val_pred)
        sensitivity = sensitivity_multiclass(cm)
        specificity = specificity_multiclass(cm)
        f1 = f1_score_multiclass(cm)
        accuracy = accuracy_multiclass(cm)
        iou = iou_multiclass(cm)


        evaluate_metrics(val_true, val_pred)

        self.sensitivities.append(sensitivity)
        self.specificities.append(specificity)
        self.f1_scores.append(f1)
        self.accuracies.append(accuracy)
        self.ious.append(iou)

        logs['val_sensitivity'] = sensitivity
        logs['val_specificity'] = specificity
        logs['val_f1'] = f1
        logs['val_accuracy'] = accuracy
        logs['val_iou'] = iou


        '''# Update logs if necessary
        logs['val_sensitivity'] = sensitivity_multiclass(confusion_matrix(val_true, val_pred))
        logs['val_specificity'] = specificity_multiclass(confusion_matrix(val_true, val_pred))
        logs['val_f1'] = f1_score_multiclass(confusion_matrix(val_true, val_pred))
        logs['val_accuracy'] = accuracy_multiclass(confusion_matrix(val_true, val_pred))
        logs['val_iou'] = iou_multiclass(confusion_matrix(val_true, val_pred))'''

        print(f" — val_sensitivity: {sensitivity} — val_specificity: {specificity} — val_f1: {f1} — val_accuracy: {accuracy} — val_iou: {iou}")

    def get_metrics(self):
        return {
            'average_sensitivity': np.mean(self.sensitivities),
            'average_specificity': np.mean(self.specificities),
            'average_f1': np.mean(self.f1_scores),
            'average_accuracy': np.mean(self.accuracies),
            'average_iou': np.mean(self.ious)
        }

def accuracy_multiclass(confusion_matrix):
    true_positives = np.diag(confusion_matrix)
    total_samples = np.sum(confusion_matrix)
    accuracy = np.sum(true_positives) / total_samples
    return accuracy

def sensitivity_multiclass(confusion_matrix):
    sensitivities = []
    for i in range(confusion_matrix.shape[0]):
        true_positives = confusion_matrix[i, i]
        false_negatives = np.sum(confusion_matrix[i, :]) - true_positives
        sensitivity = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
        sensitivities.append(sensitivity)
    return np.mean(sensitivities)

def specificity_multiclass(confusion_matrix):
    specificities = []
    for i in range(confusion_matrix.shape[0]):
        true_negatives = np.sum(np.delete(np.delete(confusion_matrix, i, axis=0), i, axis=1))
        false_positives = np.sum(np.delete(confusion_matrix[:, i], i))
        specificity = true_negatives / (true_negatives + false_positives)
        specificities.append(specificity)
    return np.mean(specificities)

def f1_score_multiclass(confusion_matrix):
    f1_scores = []
    for i in range(confusion_matrix.shape[0]):
        true_positives = confusion_matrix[i, i]
        false_positives = np.sum(confusion_matrix[:, i]) - true_positives
        false_negatives = np.sum(confusion_matrix[i, :]) - true_positives
        precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
        recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        f1_scores.append(f1)
    return np.mean(f1_scores)

def iou_multiclass(confusion_matrix):
    iou_list = []
    for i in range(confusion_matrix.shape[0]):
        true_positive = confusion_matrix[i, i]
        false_positive = np.sum(confusion_matrix[:, i]) - true_positive
        false_negative = np.sum(confusion_matrix[i, :]) - true_positive
        iou = true_positive / (true_positive + false_positive + false_negative) if (true_positive + false_positive + false_negative) > 0 else 0
        iou_list.append(iou)
    return np.mean(iou_list)

# Compute sample weights based on tile distribution
def compute_sample_weights(patient_ids):
    unique_ids, counts = np.unique(patient_ids, return_counts=True)
    total_tiles = np.sum(counts)
    sample_weights = np.array([total_tiles / count for count in counts])
    
    # Normalize the weights so they sum to 1
    sample_weights /= np.sum(sample_weights)
    
    # Create a dictionary mapping patient_id to its corresponding weight
    weight_dict = {patient_id: weight for patient_id, weight in zip(unique_ids, sample_weights)}
    
    return weight_dict

# Define weighted focal loss
def weighted_focal_loss(gamma=2.0):
    def focal_loss(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        cross_entropy = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
        prob_true = tf.reduce_sum(y_true * y_pred, axis=-1)
        weight = tf.reduce_sum(y_true, axis=-1)
        focal_loss = weight * tf.pow((1.0 - prob_true), gamma) * cross_entropy
        return tf.reduce_mean(focal_loss)
    return focal_loss


# Plotting accuracy and loss for both training and validation
def plot_training_history(history):
    # Accuracy plot
    plt.figure(figsize=(12, 5))
    
    # Accuracy plot
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    # Loss plot
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

# Cross-validation
def cross_validate(data_dir, k=5, image_size=(224, 224), batch_size=32, epochs=10):
    categories = ['01_Fat', '02_Stroma', '03_Tumor']
    patient_image_paths = gather_image_paths_and_labels(data_dir)
    patient_ids = np.array(list(patient_image_paths.keys()))
    
    kf = KFold(n_splits=k, shuffle=True, random_state=415)
    
    fold_no = 1
    all_sensitivities = []
    all_specificities = []
    all_f1_scores = []
    all_accuracies = []
    all_ious = []

    for train_index, val_index in kf.split(patient_ids):
        train_patient_ids = patient_ids[train_index]
        val_patient_ids = patient_ids[val_index]

        train_image_paths = []
        train_labels = []
        for patient_id in train_patient_ids:
            train_image_paths.extend(patient_image_paths[patient_id]['image_paths'])
            train_labels.extend([patient_image_paths[patient_id]['label']] * len(patient_image_paths[patient_id]['image_paths']))

        val_image_paths = []
        val_labels = []
        for patient_id in val_patient_ids:
            val_image_paths.extend(patient_image_paths[patient_id]['image_paths'])
            val_labels.extend([patient_image_paths[patient_id]['label']] * len(patient_image_paths[patient_id]['image_paths']))

        # Compute sample weights based on the training data
        train_weights_dict = compute_sample_weights(train_patient_ids)
        train_sample_weights = [train_weights_dict[patient_id] for patient_id in train_patient_ids]
        train_sample_weights_expanded = []
        for patient_id in train_patient_ids:
            train_sample_weights_expanded.extend([train_weights_dict[patient_id]] * len(patient_image_paths[patient_id]['image_paths']))

        train_ds = create_dataset(train_image_paths, train_labels, sample_weights=train_sample_weights_expanded, image_size=image_size, batch_size=batch_size, one_hot=True)
        val_ds = create_dataset(val_image_paths, val_labels, image_size=image_size, batch_size=batch_size, one_hot=True)
        
        # Print number of images for each set and category
        train_counts = count_images_per_category(train_image_paths, train_labels, categories)
        val_counts = count_images_per_category(val_image_paths, val_labels, categories)
        
        print(f"Fold {fold_no} - Training set counts: {train_counts}")
        print(f"Fold {fold_no} - Validation set counts: {val_counts}")
        
        # Apply data augmentation to the training dataset
        data_augmentation = tf.keras.Sequential([
            tf.keras.layers.RandomFlip("horizontal"),
            tf.keras.layers.RandomRotation(0.1),
            tf.keras.layers.RandomZoom(0.1),
            tf.keras.layers.RandomContrast(0.1)
        ])
        
        
        train_ds = train_ds.map(
            lambda x, y, w: (data_augmentation(x, training=True), y, w),
            num_parallel_calls=tf.data.AUTOTUNE
        )

        print(f"Training on fold {fold_no}...")
        with tf.device('/device:GPU:0'):
            print("tf.keras code in this scope will run on GPU")
            model = []
            model = create_your_model() 
            metrics_callback = MetricsCallback(val_ds)
            hist = model.fit(train_ds,
                             validation_data=val_ds,
                             epochs=epochs,
                             verbose=1,
                             callbacks=[metrics_callback, advanced_scheduler])
            print(f"Model_Average accuracy on fold {fold_no}: ", max(hist.history["val_accuracy"]))
            plot_training_history(hist)

            metrics = metrics_callback.get_metrics()
            all_accuracies.append(metrics['average_accuracy'])
            all_sensitivities.append(metrics['average_sensitivity'])
            all_specificities.append(metrics['average_specificity'])
            all_f1_scores.append(metrics['average_f1'])
            all_ious.append(metrics['average_iou'])

            # Define the directory and file name
            directory = 'C:/Users/39351/Documents/GitHub/Keras_Integrated_Classifier_STN_CNN/best_model'
            file_name = 'L_ViT' + str(fold_no) + '.hdf5'

            # Check if the directory exists, if not, create it
            if not os.path.exists(directory):
                os.makedirs(directory)

            p = os.path.join(directory, file_name)
            print(p)

            # Save the model
            #model.save(p, overwrite=True)
            #model.save('best_model/L_ViT'+str(fold_no)+'.hdf5', overwrite=True)
           
        
        fold_no += 1

    # Compute the average and standard deviation for each metric
    avg_accuracy = np.mean(all_accuracies)
    std_accuracy = np.std(all_accuracies)
    
    avg_sensitivity = np.mean(all_sensitivities)
    std_sensitivity = np.std(all_sensitivities)
    
    avg_specificity = np.mean(all_specificities)
    std_specificity = np.std(all_specificities)
    
    avg_f1_score = np.mean(all_f1_scores)
    std_f1_score = np.std(all_f1_scores)
    
    avg_iou = np.mean(all_ious)
    std_iou = np.std(all_ious)

    print("\nOverall Average Metrics across all folds:")
    print(f"Average Accuracy: {avg_accuracy} ± {std_accuracy}")
    print(f"Average Sensitivity: {avg_sensitivity} ± {std_sensitivity}")
    print(f"Average Specificity: {avg_specificity} ± {std_specificity}")
    print(f"Average F1 Score: {avg_f1_score} ± {std_f1_score}")
    print(f"Average IoU: {avg_iou} ± {std_iou}")

# Assuming you have a function to create your model
def create_your_model():
    mdl = []
    mdl = create_model(img_height=32, img_width=32, num_classes=3)
    #model_name = 'MobileNetV3Large'
    #mdl = create_base_model(model_name,img_height=32, img_width=32, num_classes=4)
    mdl.summary()
    #mdl.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
    mdl.compile(optimizer= optimizer, loss = weighted_focal_loss(), metrics=['accuracy'])
    return mdl

# Parameters
data_dir = 'D:/5-Histo-Cell-Dataset'  # Replace with your dataset directory
k = 3  # Number of folds
image_size = (img_width, img_height)

# Run cross-validation
cross_validate(data_dir, k, image_size, batch_size, epochs)


Fold 1 - Training set counts: {'01_Fat': 65445, '02_Stroma': 91662, '03_Tumor': 94544}
Fold 1 - Validation set counts: {'01_Fat': 34776, '02_Stroma': 68405, '03_Tumor': 31539}
Training on fold 1...
tf.keras code in this scope will run on GPU
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
rescaling_1 (Rescaling)         (None, 32, 32, 3)    0           input_2[0][0]                    
__________________________________________________________________________________________________
normalization_1 (Normalization) (None, 32, 32, 3)    7           rescaling_1[0][0]                
________________________________________________